## PhysicsSim

In [ ]:
import numpy as np


def C(x):
    return np.cos(x)


def S(x):
    return np.sin(x)


def earth_to_body_frame(ii, jj, kk):
    # C^b_n
    R = [[C(kk) * C(jj), C(kk) * S(jj) * S(ii) - S(kk) * C(ii), C(kk) * S(jj) * C(ii) + S(kk) * S(ii)],
         [S(kk) * C(jj), S(kk) * S(jj) * S(ii) + C(kk) * C(ii), S(kk) * S(jj) * C(ii) - C(kk) * S(ii)],
         [-S(jj), C(jj) * S(ii), C(jj) * C(ii)]]
    return np.array(R)


def body_to_earth_frame(ii, jj, kk):
    # C^n_b
    return np.transpose(earth_to_body_frame(ii, jj, kk))


class PhysicsSim():
    def __init__(self, init_pose=None, init_velocities=None, init_angle_velocities=None, runtime=5.):
        self.init_pose = init_pose
        self.init_velocities = init_velocities
        self.init_angle_velocities = init_angle_velocities
        self.runtime = runtime

        self.gravity = -9.81  # m/s
        self.rho = 1.2
        self.mass = 0.958  # 300 g
        self.dt = 1 / 50.0  # Timestep
        self.C_d = 0.3
        self.l_to_rotor = 0.4
        self.propeller_size = 0.1
        width, length, height = .51, .51, .235
        self.dims = np.array([width, length, height])  # x, y, z dimensions of quadcopter
        self.areas = np.array([length * height, width * height, width * length])
        I_x = 1 / 12. * self.mass * (height**2 + width**2)
        I_y = 1 / 12. * self.mass * (height**2 + length**2)  # 0.0112 was a measured value
        I_z = 1 / 12. * self.mass * (width**2 + length**2)
        self.moments_of_inertia = np.array([I_x, I_y, I_z])  # moments of inertia

        env_bounds = 300.0  # 300 m / 300 m / 300 m
        self.lower_bounds = np.array([-env_bounds / 2, -env_bounds / 2, 0])
        self.upper_bounds = np.array([env_bounds / 2, env_bounds / 2, env_bounds])

        self.reset(None)

    def reset(self, pos):
        if pos is not None:
            self.init_pose = np.array(pos)
            
        self.time = 0.0
        self.pose = np.array([0.0, 0.0, 10.0, 0.0, 0.0, 0.0]) if self.init_pose is None else self.init_pose
        self.v = np.array([0.0, 0.0, 0.0]) if self.init_velocities is None else self.init_velocities
        self.angular_v = np.array([0.0, 0.0, 0.0]) if self.init_angle_velocities is None else self.init_angle_velocities
        self.linear_accel = np.array([0.0, 0.0, 0.0])
        self.angular_accels = np.array([0.0, 0.0, 0.0])
        self.prop_wind_speed = np.array([0., 0., 0., 0.])
        self.done = False

    def find_body_velocity(self):
        body_velocity = np.matmul(earth_to_body_frame(*list(self.pose[3:])), self.v)
        return body_velocity

    def get_linear_drag(self):
        linear_drag = 0.5 * self.rho * self.find_body_velocity()**2 * self.areas * self.C_d
        return linear_drag

    def get_linear_forces(self, thrusts):
        # Gravity
        gravity_force = self.mass * self.gravity * np.array([0, 0, 1])
        # Thrust
        thrust_body_force = np.array([0, 0, sum(thrusts)])
        # Drag
        drag_body_force = -self.get_linear_drag()
        body_forces = thrust_body_force + drag_body_force

        linear_forces = np.matmul(body_to_earth_frame(*list(self.pose[3:])), body_forces)
        linear_forces += gravity_force
        return linear_forces

    def get_moments(self, thrusts):
        thrust_moment = np.array([(thrusts[3] - thrusts[2]) * self.l_to_rotor,
                            (thrusts[1] - thrusts[0]) * self.l_to_rotor,
                            0])# (thrusts[2] + thrusts[3] - thrusts[0] - thrusts[1]) * self.T_q])  # Moment from thrust

        drag_moment =  self.C_d * 0.5 * self.rho * self.angular_v * np.absolute(self.angular_v) * self.areas * self.dims * self.dims
        moments = thrust_moment - drag_moment # + motor_inertia_moment
        return moments

    def calc_prop_wind_speed(self):
        body_velocity = self.find_body_velocity()
        phi_dot, theta_dot = self.angular_v[0], self.angular_v[1]
        s_0 = np.array([0., 0., theta_dot * self.l_to_rotor])
        s_1 = -s_0
        s_2 = np.array([0., 0., phi_dot * self.l_to_rotor])
        s_3 = -s_2
        speeds = [s_0, s_1, s_2, s_3]
        for num in range(4):
            perpendicular_speed = speeds[num] + body_velocity
            self.prop_wind_speed[num] = perpendicular_speed[2]

    def get_propeler_thrust(self, rotor_speeds):
        '''calculates net thrust (thrust - drag) based on velocity
        of propeller and incoming power'''
        thrusts = []
        for prop_number in range(4):
            V = self.prop_wind_speed[prop_number]
            D = self.propeller_size
            n = rotor_speeds[prop_number]
            J = V / n * D
            # From http://m-selig.ae.illinois.edu/pubs/BrandtSelig-2011-AIAA-2011-1255-LRN-Propellers.pdf
            C_T = max(.12 - .07*max(0, J)-.1*max(0, J)**2, 0)
            thrusts.append(C_T * self.rho * n**2 * D**4)
        return thrusts

    def next_timestep(self, rotor_speeds):
        self.calc_prop_wind_speed()
        thrusts = self.get_propeler_thrust(rotor_speeds)
        self.linear_accel = self.get_linear_forces(thrusts) / self.mass

        position = self.pose[:3] + self.v * self.dt + 0.5 * self.linear_accel * self.dt**2
        self.v += self.linear_accel * self.dt

        moments = self.get_moments(thrusts)

        self.angular_accels = moments / self.moments_of_inertia
        angles = self.pose[3:] + self.angular_v * self.dt + 0.5 * self.angular_accels * self.angular_accels * self.dt
        angles = (angles + 2 * np.pi) % (2 * np.pi)
        self.angular_v = self.angular_v + self.angular_accels * self.dt

        new_positions = []
        for ii in range(3):
            if position[ii] <= self.lower_bounds[ii]:
                new_positions.append(self.lower_bounds[ii])
                self.done = True
            elif position[ii] > self.upper_bounds[ii]:
                new_positions.append(self.upper_bounds[ii])
                self.done = True
            else:
                new_positions.append(position[ii])

        self.pose = np.array(new_positions + list(angles))
        self.time += self.dt
        if self.time > self.runtime:
            self.done = True
        return self.done

## Actor / Critic models

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from ipdb import set_trace as debug

def fanin_init(size, fanin=None):
    fanin = fanin or size[0]
    v = 1. / np.sqrt(fanin)
    return torch.Tensor(size).uniform_(-v, v)

class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=400, hidden2=300, init_w=3e-3):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, nb_actions)
        self.relu = nn.ReLU()
        #self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        #self.softmax = nn.Softmax()
        self.init_weights(init_w)
    
    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-init_w, init_w)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=400, hidden2=300, init_w=3e-3):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(nb_states, hidden1)
        self.fc2 = nn.Linear(hidden1+nb_actions, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
        self.relu = nn.ReLU()
        self.init_weights(init_w)
    
    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-init_w, init_w)
    
    def forward(self, xs):
        x, a = xs
        out = self.fc1(x)
        out = self.relu(out)
        # debug()
        out = self.fc2(torch.cat([out,a],1))
        out = self.relu(out)
        out = self.fc3(out)
        return out

## Utils

In [ ]:
import numpy as np
import torch
import random
from collections import namedtuple, deque
from torch.autograd import Variable

USE_CUDA = torch.cuda.is_available()
FLOAT = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor

class ExperienceMemory:
    """Experience Memory Buffer"""

    def __init__(self, buffer_size, batch_size):
        """Initialize buffer object."""
        self.memory = deque(maxlen=buffer_size)  # fifo queue
        self.batch_size = batch_size
        self.general_feeling = 0.0 # not used yet :)
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        self.action_size = np.array(action).shape[0]
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
        
    def sweet_dreams(self, batch_size=64):
        """Recall and replay experiences in memory, reordering experiences by positive reward."""
        for e in sorted(self.memory,key=lambda x: x.reward, reverse=False):
            self.memory.append(e)
    
    def sample(self, batch_size=64):
        """Randomly sample a batch of experiences from memory."""
        return random.sample(self.memory, k=self.batch_size)
    
    def batch_samples(self, batch_size=64):
        """"""
        exp = self.sample(batch_size=64)
        
        states = np.vstack([e.state for e in exp if e is not None])
        actions = np.array([e.action for e in exp if e is not None]).astype(np.float32).reshape(-1, self.action_size)
        
        rewards = np.array([e.reward for e in exp if e is not None]).astype(np.float32).reshape(-1, 1)
        next_states = np.vstack([e.next_state for e in exp if e is not None])
        dones = np.array([e.done for e in exp if e is not None]).astype(np.uint8).reshape(-1, 1)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)
    

    
class OUNoise:
    """
    Ornstein-Uhlenbeck process.
    https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process
    """

    def __init__(self, size, mu, theta, sigma):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.state = self.mu

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(len(x))
        self.state = x + dx
        return self.state
    
    def reset(self):
        self.state = self.mu
        
def to_numpy(var):
    return var.cpu().data.numpy() if USE_CUDA else var.data.numpy()

def to_tensor(ndarray, requires_grad=False, dtype=FLOAT):
    return Variable(
        torch.from_numpy(ndarray), requires_grad=requires_grad
    ).type(dtype)

## DDPG Agent

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.optim import Adam

# from ipdb import set_trace as debug

criterion = nn.MSELoss()

class Agent(object):
    def __init__( self, task, hp ):
                   
        self.task = task
        self.nb_states = task.state_size
        self.nb_actions = task.action_size
        self.action_low = task.action_low
        self.action_high = task.action_high
        self.action_range = self.action_high - self.action_low
        
        self.use_cuda = 1 if hp['USE_CUDA'] is True else 0
        
        if int(hp['SEED']) > 0:
            self.seed(hp['SEED'])
        
        self.buffer_size = hp['EXP_BUFFER_SIZE']
        self.batch_size = hp['EXP_BATCH_SIZE']

        
        # Create Actor and Critic Network
        net_cfg = {
            'hidden1':hp['HIDDEN1'], 
            'hidden2':hp['HIDDEN2'], 
            'init_w':hp['INIT_W']
        }
        self.actor = Actor(self.nb_states, self.nb_actions, **net_cfg)
        self.actor_target = Actor(self.nb_states, self.nb_actions, **net_cfg)
        self.actor_optim  = Adam(self.actor.parameters(), lr=hp['ACTOR_LR'])

        self.critic = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_target = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_optim  = Adam(self.critic.parameters(), lr=hp['CRITIC_LR'])
            
        self.hard_copy( self.actor, self.actor_target ) 
        self.hard_copy( self.critic, self.critic_target )
        
        # Create experience memory buffer
        self.memory = ExperienceMemory(self.buffer_size, self.batch_size)
        
        # init the process of life ... ..
        self.random_process = OUNoise(size=self.nb_actions, theta=hp['OU_THETA'], mu=hp['OU_MU'], sigma=hp['OU_SIGMA'])
        self.ou_decay = hp['OU_DECAY']

        # Hyper-parameters
        #self.batch_size = hp.BATCH_SIZE
        self.tau = hp['TAU']
        self.discount = hp['DISCOUNT']
        #self.depsilon = 1.0 / args.epsilon

        # 
        #self.epsilon = 1.0
        self.s_t = None # Most recent state
        self.a_t = None # Most recent action
        self.is_training = True

        # nvidia
        if hp['USE_CUDA']: 
            self.cuda()
            
    def hard_copy(self, source, target):
        for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)
            
    def soft_update(self, source, target):
        for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_( target_param.data * (1.0 - self.tau) + param.data * self.tau )


    def update_policy(self):
        
        # Get Sample batches
        state_batch, action_batch, reward_batch, \
        next_state_batch, terminal_batch = self.memory.batch_samples(self.batch_size)

        ###########################################
        # Prepare for the target q batch
        with torch.no_grad():
            next_q_values = self.critic_target([
                to_tensor(next_state_batch),
                self.actor_target(to_tensor(next_state_batch)),
            ])
        #next_q_values.volatile=False

        target_q_batch = to_tensor(reward_batch) + \
            self.discount*to_tensor(terminal_batch.astype(np.float))*next_q_values

        ############################################
        # Critic update
        self.critic.zero_grad()

        q_batch = self.critic([ to_tensor(state_batch), to_tensor(action_batch) ])
        
        value_loss = criterion(q_batch, target_q_batch)
        value_loss.backward()
        self.critic_optim.step()

        ##############################################
        # Actor update
        self.actor.zero_grad()

        policy_loss = -self.critic([
            to_tensor(state_batch),
            self.actor(to_tensor(state_batch))
        ])

        policy_loss = policy_loss.mean()
        policy_loss.backward()
        self.actor_optim.step()

        ###############################################
        # Target update
        self.soft_update( self.actor, self.actor_target )
        self.soft_update( self.critic, self.critic_target )

        
    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()
        self.is_training = False

        
    def cuda(self):
        self.use_cuda = 1
        self.actor.cuda()
        self.actor_target.cuda()
        self.critic.cuda()
        self.critic_target.cuda()
        
        
    def step(self, action, reward, next_state, done):
         # Save experience / reward
        self.a_t = action
        self.observe( reward, next_state, done )

        # If we got our minibatch of experience memories..
        # learn from them and slowly change belief :)
        if len(self.memory) > self.batch_size:           
            self.update_policy()

    def observe(self, r_t, s_t1, done):
        if self.is_training:
            self.memory.add(self.s_t, self.a_t, r_t, s_t1, done)
            self.s_t = s_t1

    def random_action(self):
        action = np.random.uniform(-1.,1.,self.nb_actions)
        self.a_t = action
        return action
    
    def act(self, s_t, i_eposode, decay_epsilon=True):
        
        action = to_numpy(
            self.actor(to_tensor(np.array([s_t])))
        ).squeeze(0)
        
        action = (action * self.action_range) + self.action_low
        
        if(self.ou_decay != 0):
            decay = 1 - (i_episode*self.ou_decay)
            action += self.is_training*decay*self.random_process.sample()
        
        self.a_t = action
        return action

    def reset(self, obs):
        self.s_t = self.task.reset(obs)
        self.random_process.reset()
        return self.s_t

    def load_weights(self, output):
        if output is None: return

        self.actor.load_state_dict(
            torch.load('{}/actor.pkl'.format(output))
        )

        self.critic.load_state_dict(
            torch.load('{}/critic.pkl'.format(output))
        )


    def save_model(self,output):
        torch.save(
            self.actor.state_dict(),
            '{}/actor.pkl'.format(output)
        )
        torch.save(
            self.critic.state_dict(),
            '{}/critic.pkl'.format(output)
        )

    def seed(self,s):
        torch.manual_seed(s)
        if self.use_cuda:
            torch.cuda.manual_seed(s)


## Tasks

In [ ]:
import numpy as np
#from physics_sim import PhysicsSim

class Hover():
    """Task (environment) that defines the goal and provides feedback to the agent."""
    def __init__(self, init_pose=None, init_velocities=None, 
        init_angle_velocities=None, runtime=5., target_pos=None):
        """Initialize a Task object.
        Params
        ======
            init_pose: initial position of the quadcopter in (x,y,z) dimensions and the Euler angles
            init_velocities: initial velocity of the quadcopter in (x,y,z) dimensions
            init_angle_velocities: initial radians/second for each of the three Euler angles
            runtime: time limit for each episode
            target_pos: target/goal (x,y,z) position for the agent
        """
        # Simulation
        self.sim = PhysicsSim(init_pose, init_velocities, init_angle_velocities, runtime) 
        self.action_repeat = 3

        self.state_size = self.action_repeat * (len(self.sim.pose))
        self.action_low = 0
        self.action_high = 900
        self.action_size = 4
        self.prev_distance = None

        # Goal
        self.target_pos = target_pos if target_pos is not None else np.array([0., 0., 10.]) 

    def get_reward(self):
        """Uses current pose of sim to return reward."""
        #print("POSITION:\n  {:3.4f},{:3.4f},{:3.4f}".format(self.sim.pose[0],self.sim.pose[1],self.sim.pose[2]))
        sum_distance = (abs(self.sim.pose[:3] - self.target_pos)).sum()
        # D = √[(x₂ - x₁)² + (y₂ - y₁)² +(z₂ - z₁)²]
        distance = np.linalg.norm(self.sim.pose[:3] - self.target_pos)
        #x0,y0,z0 = self.target_pos[:3]
        #x1,y2,z1 = self.sim.pose[:3] 
    
        reward = 1.- .003*sum_distance
        if sum_distance < 0.4:
            return 100
        #reward = (10.+self.sim.time) - ( distance * (abs(self.sim.v)).sum() )
        #max_xy_angle = max(abs(self.sim.pose[3]),abs(self.sim.pose[4]))
        
        # we want to "stay" ..so we use every signal that we are not staying as sum penalty
        #sum_distance = (abs(self.sim.pose[:3] - self.target_pos)).sum()
        #sum_angle = abs(self.sim.pose[3:]).sum()
        #sum_velocity = abs(self.sim.v).sum()
        #penalty = (sum_distance+(sum_velocity*0.01)) # + sum_velocity + sum_angle)
        #reward = (1+self.sim.time) - (distance + (angle / 4))
        #reward = 25 - penalty

        #reward = 10 - distance # + (sum_angle*0.025)
        #reward = reward * 0.1
        
        #punishment = -( distance + sum_angle + sum_velocity)
        
        #reward  = 10. - (distance * 0.1)
        #if self.prev_distance == None or self.prev_distance == distance:
        #    reward = 10.0 - (distance * 0.1)
        #elif self.prev_distance > distance:
        #    reward = 20.0 - (distance * 0.1)
        #elif self.prev_distance < distance:
        #    reward = -20.0 - (distance * 0.1)
            
        self.prev_distance = distance
        return reward

    def step(self, rotor_speeds):
        """Uses action to obtain next state, reward, done."""
        reward = 0
        pose_all = []
        done = False
        #print("Taking repeated steps:")
        for _ in range(self.action_repeat):
            done = self.sim.next_timestep(rotor_speeds) # update the sim pose and velocities
            reward += self.get_reward()
            if reward == 100:
                done = 1
            #if reward <= -3.0:
            #    done = True
            #if done and self.sim.time < self.sim.runtime:
            #    reward = -100

            #print(self.sim.pose)
            #distances = np.array(self.sim.pose[:3] - self.target_pos)
            #print(distances)
            #print(np.concatenate([self.sim.pose,distances]))
            #pose_all.append(np.concatenate([self.sim.pose,distances]))
            pose_all.append(np.concatenate([self.sim.pose]))
        next_state = np.concatenate(pose_all)
        return next_state, reward, done

    def reset(self, pos):
        """Reset the sim to start a new episode."""
        self.sim.reset(pos)
        distances = np.array(self.sim.pose[:3] - self.target_pos)
        state = np.concatenate([self.sim.pose]) 
        state = np.concatenate([state] * self.action_repeat) 
        return state

## Define hyperparameters and initialize

In [ ]:
from collections import namedtuple

hparams = {
    'EPISODES'       : 2000,
    'EP_MAX_RUNTIME' : 20,
    'SEED'           : 0,
    'USE_CUDA'       : False,
    # deep nn
    'ACTOR_LR'    : 0.0001, # adjustment per backprop
    'CRITIC_LR'   : 0.001, # adjustment per backprop
    'INIT_W'      : 0.003, # init weight distribution size
    'HIDDEN1'     : 16, 
    'HIDDEN2'     : 16,
#    'ACTOR_L2_RR' : 0.01, # L2/ridge regularization
#    'CRITIC_L2_RR': 0.01, # L2/ridge regularization
    'TAU'         : 0.001, # The factor of convergence for target nets
    'DROPOUT'     : 0.25, # Neuron dropout-factor (not implemented atm)
    
    # Ornstein-Uhlenbeck Exploration
    'OU_MU'    : 0.0,   # noise overall offset
    'OU_THETA' : .15,  # how “fast” the variable reverts towards to the mean
    'OU_SIGMA' : .3, # scale factor
    'OU_DECAY' : 1.5, # zero to disable OU
    
    # Q-learning
    'DISCOUNT' : .99,  # value discount factor

    # Instead of pixels we use a batch of environmental data / states 
    # (experiences), that we feed to our deep neural networks
    'EXP_BUFFER_SIZE' : 8000000,
    'EXP_BATCH_SIZE' : 100 # 
}
# create a decay
if hparams['OU_DECAY'] != 0:
    hparams['OU_DECAY'] = 1 / ( hparams['EPISODES'] * hparams['OU_DECAY'] )

#hparams = namedtuple("HyperParams",hparams)

init_pose = np.array([.5, .5, 99.5, 0., 0., 0.])
target_pos = np.array([0., 0., 100.])
#task = Task(runtime=2.5, target_pos=target_pos)
task = Hover(runtime=hparams['EP_MAX_RUNTIME'], init_pose=init_pose, target_pos=target_pos)
agent = Agent(task, hparams) 
print(agent.actor)
print(agent.critic)

## Run age..*cough*  Fly Agent ..Fly!

In [ ]:
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
%matplotlib inline

num_episodes = hparams['EPISODES']

worst_score = 0
best_score = 0

reward_labels = ['episode', 'reward']
reward_results = {x : [] for x in reward_labels}

starting_positions = [
    [0., 0, 95.],
    [0., 0, 105.]
]

for i_episode in range(1, num_episodes+1):
    ep_score = 0
    #x = np.random.uniform(-2.,2.)
    #y = np.random.uniform(-2.,2.)
    #z = np.random.uniform(98.,102.)
    #i = random.sample(starting_positions,len(starting_positions)-1)
    pos = starting_positions[0]#i_episode%2]
    state = agent.reset([pos[0], pos[1], pos[2], 0., 0., 0.])
    #state = agent.reset_episode(starting_positions[i_episode%2])

    pose_bucket = {'x':[],'y':[],'z':[]}
    rotor = {'time':[],'1':[],'2':[],'3':[],'4':[]}
    rewards = []
    
    while True:
        action = agent.act(state, i_episode) 
        next_state, reward, done = task.step(action)
        agent.step(action, reward, next_state, done)
        state = next_state
        ep_score += reward
        best_score = max(best_score , ep_score)
        worst_score = min(worst_score , ep_score)
        x,y,z = task.sim.pose[:3]
        pose_bucket['x'].append(x)
        pose_bucket['y'].append(y)
        pose_bucket['z'].append(z)
        rewards.append(reward)
        rotor['time'].append(task.sim.time)
        rotor['1'].append(action[0])
        rotor['2'].append(action[1])
        rotor['3'].append(action[2])
        rotor['4'].append(action[3])
        
        
        if done or reward <= -1000:
            #clear_output(wait=True)
            f = plt.figure(figsize=(15,6))
            ax1 = f.add_subplot(2, 2, 1)
            #plt.tight_layout()
            ax1.plot(rewards)
            #ax = fig.gca(projection='3d')
            ax2 = f.add_subplot(2,2,2,projection='3d')
            ax2.plot(pose_bucket['x'], pose_bucket['y'], pose_bucket['z'], label='quadraview')
            ax2.set_zlim3d(0, 300)
            ax2.set_xlim(-300, 300)
            ax2.set_ylim(-300, 300)
            
            ax3 = f.add_subplot(2,2,3)
            ax3.plot(rotor['time'], rotor['1'], label='r1')
            ax3.plot(rotor['time'], rotor['2'], label='r2')
            ax3.plot(rotor['time'], rotor['3'], label='r3')
            ax3.plot(rotor['time'], rotor['4'], label='r4')
            
            #ax.quiver(pose_bucket['x'], pose_bucket['y'], pose_bucket['z'], 0.0, 0.0, 0.0, label='quadraview')
            plt.show()
            avg_score = ep_score / len(rewards)
            print("Episode = {:4d}, score = {:7.3f} (best = {:7.3f} , worst = {:7.3f})".format(
               i_episode, avg_score, ep_score, best_score, worst_score), end="\n")
            break
    reward_results['episode'].append(i_episode)
    reward_results['reward'].append(ep_score/len(rewards))
    sys.stdout.flush()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(reward_results['episode'], reward_results['reward'], label='avg reward / episode')
plt.legend()
_ = plt.ylim()

In [ ]:
import csv

runtime = 150.                                     # time limit of the episode
init_pose = np.array([0., 0., 100., 0., 0., 0.])  # initial pose
init_velocities = np.array([0., 0., 0.])         # initial velocities
init_angle_velocities = np.array([0., 0., 0.])   # initial angle velocities
target_pos = np.array([0., 0., 100.])   


task = Hover(init_pose, init_velocities, init_angle_velocities, runtime, target_pos)
#task = Hover( init_pose, target_pos, runtime=runtime, target_pos=target_pos )
done = False
labels = ['time', 'x', 'y', 'z', 'phi', 'theta', 'psi', 'x_velocity',
          'y_velocity', 'z_velocity', 'phi_velocity', 'theta_velocity',
          'psi_velocity', 'rotor_speed1', 'rotor_speed2', 'rotor_speed3', 'rotor_speed4']
results = {x : [] for x in labels}


agent.eval()
state = agent.reset(None)
total_reward = 0
while True:
    rotor_speeds = agent.act(state, 1)
    print(rotor_speeds)
    next_state, reward, done = task.step(rotor_speeds)
    to_write = [task.sim.time] + list(task.sim.pose) + list(task.sim.v) + list(task.sim.angular_v) + list(rotor_speeds)
    for ii in range(len(labels)):
        results[labels[ii]].append(to_write[ii])
    total_reward += reward
    state = next_state
    if done:
        print("Total episode reward : {}".format(total_reward))
        total_reward = 0
        break

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(results['time'], results['rotor_speed1'], label='r1')
plt.plot(results['time'], results['rotor_speed2'], label='r2')
plt.plot(results['time'], results['rotor_speed3'], label='r3')
plt.plot(results['time'], results['rotor_speed4'], label='r4')
plt.legend()
_ = plt.ylim()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(results['time'], results['phi'], label='phi')
plt.plot(results['time'], results['theta'], label='theta')
plt.plot(results['time'], results['psi'], label='psi')
plt.legend()
_ = plt.ylim()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(results['time'], results['x'], label='x')
plt.plot(results['time'], results['y'], label='y')
plt.plot(results['time'], results['z'], label='z')
plt.legend()
_ = plt.ylim()